# 데이터 로드

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_item_categories = pd.read_csv("item_categories.csv")
df_items = pd.read_csv("items.csv")
df_sales_train = pd.read_csv("sales_train.csv")
df_shops = pd.read_csv("shops.csv")
df_test = pd.read_csv("test.csv")

# 데이터 분석

In [3]:
df_sales_train.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
df_test.head(5)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
df_sales_train["item_cnt_day"].value_counts().head(10)

 1.0    2629372
 2.0     194201
 3.0      47350
 4.0      19685
 5.0      10474
-1.0       7252
 6.0       6338
 7.0       4057
 8.0       2903
 9.0       2177
Name: item_cnt_day, dtype: int64

## 판매량을 예측해달라.
근데 어떻게?
한달단위로 데이터를 끊어줬음.
그다음, 테스트데이터를 보면 shop_id, item_id밖에없다 - > 날짜 없음.
그럼 이것도 그냥 한달 종합데이터로 여겨버리면된다. -> 우리가 가진 훈련데이터를 날짜를 지우고 한달로 다 묶어버리면 됨.
이 방식은 시계열값을 날리고 하기때문에 시계열 분석이 아니게된다.

그럼 시계열분석을 어떻게해야할까? -> 시간에 따른 추세를 분석한다는 것.
시계열분석에 여러가지가있어
통계적기법 -> ARIMA
인공지능 -> RNN GRU LSTM 1DCNN


In [6]:
df_sales_train = df_sales_train.drop(['date'],axis=1)

In [7]:
df_sales_train.head(5)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1.0
1,0,25,2552,899.00,1.0
2,0,25,2552,899.00,-1.0
3,0,25,2554,1709.05,1.0
4,0,25,2555,1099.00,1.0


In [8]:
m = 0
for key, values in df_sales_train.groupby(df_sales_train["date_block_num"]):
    globals()['Month_{}'.format(m)] = values
    m +=1

In [9]:
Month_0

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1.0
1,0,25,2552,899.00,1.0
2,0,25,2552,899.00,-1.0
3,0,25,2554,1709.05,1.0
4,0,25,2555,1099.00,1.0
...,...,...,...,...,...
115685,0,46,32,349.00,1.0
115686,0,46,32,349.00,1.0
115687,0,46,35,399.00,2.0
115688,0,46,621,149.00,1.0


In [10]:
df_sales_train = df_sales_train.drop(["date_block_num","item_price"], axis=1)

In [11]:
df_sales_train

,shop_id,item_id,item_cnt_day
0,59,22154,1.0
1,25,2552,1.0
2,25,2552,-1.0
3,25,2554,1.0
4,25,2555,1.0
...,...,...,...
2935844,25,7409,1.0
2935845,25,7460,1.0
2935846,25,7459,1.0
2935847,25,7440,1.0


# X가 두개. Y는 1개. Y의 값은 다양
어떤 모델을 써야할까
Classification, Clustering, Regression~~~~~

Classification -> 분류 모델
분류 -> 0,1 이정도로 작은 수의 Y값의 종류를 가졌을때 단순 분류하는거를 말함. 00000 / 111111

Clustering
Y이 값이 Classification보단 다양함.
Y에 대해 군집화 시킴.

Regression
회귀
분포들을 대표하는 함수를 만듬.
즉 정말 다양한 Y값을 가질수있다.
Y값이 연속적일수도있다.

3번째거로 가야함. 왜?
몇개팔릴지 -> 다양한 y값을 가진다. 몇개? 라는것은 다양한 y값, 또한 연속적이 가능. OK?

# 시계열 회귀분석 -> 시계열 이미 날림

In [24]:
import tensorflow as tf

In [31]:
X_train = df_sales_train.drop(["item_cnt_day"],axis=1)
X_train

(2935849, 2)

In [26]:
y_train = df_sales_train["item_cnt_day"]
y_train

0          1.0
1          1.0
2         -1.0
3          1.0
4          1.0
          ... 
2935844    1.0
2935845    1.0
2935846    1.0
2935847    1.0
2935848    1.0
Name: item_cnt_day, Length: 2935849, dtype: float64

In [27]:
X_test = df_test.drop(["ID"],axis=1)
X_test

,shop_id,item_id
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757
214198,45,19648


In [59]:
X = tf.keras.layers.Input(shape=(2,))
Y = tf.keras.layers.Dense(1)(X)
opt = tf.keras.optimizers.Adagrad(lr=0.01, epsilon=1e-6)

model = tf.keras.models.Model(X, Y)
model.compile(loss='mse',optimizer=opt)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=2, verbose=1)
print(history.history)

Epoch 1/2
68809/68809 [==============================] - 44s 631us/step - loss: 3289821.5863 - val_loss: 249.8124
Epoch 2/2
68809/68809 [==============================] - 42s 606us/step - loss: 235.5433 - val_loss: 221.3834
{'loss': [628996.75, 228.0328369140625], 'val_loss': [249.8124237060547, 221.38340759277344]}


In [49]:
y_pred = model.predict(X_test)

In [50]:
y_pred

array([[1.23798  ],
       [1.2408131],
       [1.2399422],
       ...,
       [1.305524 ],
       [1.3444777],
       [1.1574782]], dtype=float32)

In [51]:
data1 = pd.DataFrame(df_test["ID"])
data2 = pd.DataFrame(y_pred)
data=pd.concat([data1,data2], axis=1)

In [52]:
data

,ID,0
0,0,1.237980
1,1,1.240813
2,2,1.239942
3,3,1.239932
4,4,1.240293
...,...,...
214195,214195,1.332524
214196,214196,1.309839
214197,214197,1.305524
214198,214198,1.344478


In [53]:
data.to_excel(excel_writer='result.xlsx')